# Forward pass simple regression

In [ ]:
import os 
import getopt
import sys

import random
import re

import numpy as np
import h5py
import pickle
import random
import copy
import pandas as pd
import math 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Masking, Lambda, concatenate, Bidirectional, Dense, Dropout, Flatten, Conv1D,BatchNormalization,  MaxPooling1D, Bidirectional, GRU, TimeDistributed
import tensorflow as tf
from tensorflow import keras



In [ ]:
model_path = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/test_background.h5"

# For the new data I am using training data just as an example
new_data = ""/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/background_LibA_wide_pivot_state3_train_predicted_cv10fold.csv""

# Prepare input data for deep explainer
df = pd.read_csv(new_data)

def one_hot_encode_sequences(sequences, max_length, vocab):
    """One hot encodes a sequence of characters

    Args:
        sequences (np.array): Input sequences 
        vocab (list): list of vocabulary characters

    Returns:
        (np.array): one-hot-encoded array
    """
    num_sequences = len(sequences)
    max_seq_length = max_length
    encoding = np.zeros((num_sequences, max_seq_length, len(vocab)), dtype=int)

    # Create a dictionary to map characters to their corresponding indices in the vocab
    char_to_index = {char: i for i, char in enumerate(vocab)}

    for i, sequence in enumerate(sequences):
        for j, base in enumerate(sequence):
            if base in char_to_index:
                index = char_to_index[base]
                encoding[i, j, index] = 1
            else:
                # Set the entire vector to [0, 0, 0, 0] for unexpected characters
                encoding[i, j, :] = [0, 0, 0, 0]

    return encoding

vocab = ["a", "c", "g", "t"]
max_length = df.seq.str.len().max()
X = one_hot_encode_sequences(df.seq.values, max_length,vocab)
y = df.State_3E.values
ids = df.CRS.values

In [ ]:

class CustomNormalization(Layer):
    def __init__(self, **kwargs):
        super(CustomNormalization, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Add trainable variables for mean and standard deviation
        self.mean = self.add_weight("mean", shape=(1,), initializer="zeros", trainable=True)
        self.stddev = self.add_weight("stddev", shape=(1,), initializer="ones", trainable=True)
        super(CustomNormalization, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        # Normalize the inputs using the learned mean and standard deviation
        return (inputs - self.mean) / (self.stddev + 1e-8)

inputs = Input(shape=(X.shape[1], X.shape[2]), name="inputs")
#inputs = Masking()(inputs)
layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
layer = Dropout(0.3)(layer)
layer = BatchNormalization()(layer)
layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
layer = BatchNormalization()(layer)
layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
layer = Dropout(0.3)(layer)
layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
layer = BatchNormalization()(layer)
layer = Dropout(0.3)(layer)
layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
layer = BatchNormalization()(layer)
layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
layer = Dropout(0.3)(layer)
layer = Flatten()(layer)
layer = Dense(300, activation='sigmoid')(layer)  # 300
layer = Dropout(0.3)(layer)
layer = Dense(200, activation='sigmoid')(layer)  # 300
predictions = Dense(1, activation='linear')(layer)
norm_predictions = CustomNormalization()(predictions)  # Assuming "predictions" is your existing output

model = Model(inputs=inputs, outputs=norm_predictions)

model = Model(inputs=inputs, outputs=norm_predictions)
model.summary()
model.load_weights(model_path)

In [ ]:
model.predict(X)

# Forward pass multihead regression

In [ ]:

class CustomNormalization(Layer):
    def __init__(self, **kwargs):
        super(CustomNormalization, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Add trainable variables for mean and standard deviation
        self.mean = self.add_weight("mean", shape=(1,), initializer="zeros", trainable=True)
        self.stddev = self.add_weight("stddev", shape=(1,), initializer="ones", trainable=True)
        super(CustomNormalization, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        # Normalize the inputs using the learned mean and standard deviation
        return (inputs - self.mean) / (self.stddev + 1e-8)

inputs = Input(shape=(X.shape[1], X.shape[2]), name="inputs")
#inputs = Masking()(inputs)
layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
layer = Dropout(0.3)(layer)
layer = BatchNormalization()(layer)
layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
layer = BatchNormalization()(layer)
layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
layer = Dropout(0.3)(layer)
layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
layer = BatchNormalization()(layer)
layer = Dropout(0.3)(layer)
layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
layer = BatchNormalization()(layer)
layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
layer = Dropout(0.3)(layer)
layer = Flatten()(layer)
layer = Dense(300, activation='sigmoid')(layer)  # 300
layer = Dropout(0.3)(layer)
layer = Dense(200, activation='sigmoid')(layer)  # 300
predictions = Dense(1, activation='linear')(layer)
norm_predictions = CustomNormalization()(predictions)  # Assuming "predictions" is your existing output

model = Model(inputs=inputs, outputs=norm_predictions)

model = Model(inputs=inputs, outputs=norm_predictions)
model.summary()
model.load_weights(model_path)

model.predict(X)